In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
df = pd.read_excel('/content/drive/MyDrive/제주학회2(별)/실험/데이터전처리/관세용어사전_길이순.xlsx', index_col = 0)

In [5]:
df

,단어이름,단어 설명
0,수입화물대도,일람불신용장거래 추심결제방식의 거래에서 수입자가 수입어음대금을 결제하기 전이라도 선...
1,지정신용장,신용장상에 지급 인수 매입 등을 담당할 특정은행의 명칭을 미리 지정해 놓은 신용장을...
2,선하증권,화물이 본선에 적재되었음을 증명하는 서류로서 법적으로 화물 그 자체를 대표하는 양도...
3,조회,라고도 함 수입업자가 조회서한을 보낸 때에는 통상 수입희망품목의 가격 수량 선적시기...
4,특정성,보조금 및 상계조치에 관한 협정에서 사용되는 개념으로 어떤 보조금이 특정성이 있는 ...
...,...,...
595,사정 평가,조사심사하여 결정하는 것
596,습득물,잃어버린 물건 찾아주기
597,유한책임,채무자의 제한된 책임
598,행정절차법,행정절차에 관한 법률


In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.3 MB/s eta 0:00:00


In [7]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import tqdm
import time
import numpy as np

In [12]:
from transformers import AutoTokenizer, AutoModel 

In [13]:
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)

In [14]:
model = AutoModel.from_pretrained("snunlp/KR-Medium")

Some weights of the model checkpoint at snunlp/KR-Medium were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(20000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [16]:
max_len = 300

In [17]:
def tokenizer_(text, max_len, tokenizer):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    input_id = encoded_dict['input_ids']
    token_type_id = encoded_dict['token_type_ids']
    attention_mask = encoded_dict['attention_mask']
    
    return input_id, token_type_id, attention_mask

In [18]:
def bert_embedding(text):
    encoded_dict = tokenizer.encode_plus(text = text,
                                     add_special_tokens = True,
                                     max_length = max_len,
                                     padding = 'max_length',
                                     return_attention_mask = True,
                                     truncation = True)
    
    tokens_tensor = torch.tensor([encoded_dict['input_ids']]).to('cuda')  
    segment_tensors = torch.tensor([encoded_dict['token_type_ids']]).to('cuda')
    attention_tensors = torch.tensor([encoded_dict['attention_mask']]).to('cuda')
    
    model.eval()

    with torch.no_grad():
        outputs = model(tokens_tensor, attention_tensors, segment_tensors)
        
    hidden_states = outputs[0]  #[0]일 경우 last_hidden_states, [1]일 경우 pooler 출력
    
    last_hidden_states_cls = hidden_states[0][0]
    
    return last_hidden_states_cls.cpu().numpy()

In [21]:
# 단어 임베딩
krbert_embed_voc = df['단어이름'].apply(lambda x: bert_embedding(x))

In [22]:
krbert_embedding_voc = np.array(krbert_embed_voc.to_list())

In [ ]:
# np.save("/content/drive/MyDrive/제주학회3(별)/krbert_임베딩/krbert_voc.npy",krbert_embedding_voc)

In [23]:
# 정의 임베딩
krbert_embed_sen = df['단어 설명'].apply(lambda x: bert_embedding(x))

In [24]:
krbert_embedding_sen = np.array(krbert_embed_sen.to_list())

In [ ]:
# np.save("/content/drive/MyDrive/제주학회3(별)/krbert_임베딩/krbert_sen.npy",krbert_embedding_sen)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

cos_dis = cosine_similarity(krbert_embedding_voc, krbert_embedding_sen)

In [26]:
cos_dis

array([[0.62113166, 0.6077541 , 0.57676387, ..., 0.69819534, 0.6950974 ,
        0.7672986 ],
       [0.6899847 , 0.7061868 , 0.6232155 , ..., 0.78360665, 0.75800645,
        0.7417743 ],
       [0.6469983 , 0.64184487, 0.57492346, ..., 0.747146  , 0.70708716,
        0.7569686 ],
       ...,
       [0.706813  , 0.68745756, 0.64691937, ..., 0.9176797 , 0.7788604 ,
        0.79127336],
       [0.6678048 , 0.6646408 , 0.6367488 , ..., 0.82891214, 0.93142563,
        0.7741507 ],
       [0.5834919 , 0.57482374, 0.5379813 , ..., 0.77593553, 0.73328507,
        0.86033   ]], dtype=float32)

In [34]:
np.min(cos_dis[0])

0.5124595

In [27]:
cos_dis_same = []

for i in range(len(cos_dis)):
    same = cos_dis[i][i]
    #print(same)
    cos_dis_same.append([i, same])

In [28]:
cos_dis_same

[[0, 0.62113166],
 [1, 0.7061868],
 [2, 0.57492346],
 [3, 0.6018763],
 [4, 0.6146351],
 [5, 0.61408097],
 [6, 0.5875434],
 [7, 0.690953],
 [8, 0.7055202],
 [9, 0.5815496],
 [10, 0.7668239],
 [11, 0.71911734],
 [12, 0.6835341],
 [13, 0.7594619],
 [14, 0.73036146],
 [15, 0.6771074],
 [16, 0.6708588],
 [17, 0.5622325],
 [18, 0.6559299],
 [19, 0.79908395],
 [20, 0.63234645],
 [21, 0.6860304],
 [22, 0.6440118],
 [23, 0.68954784],
 [24, 0.6046357],
 [25, 0.6671114],
 [26, 0.64946973],
 [27, 0.7335271],
 [28, 0.6775263],
 [29, 0.71108824],
 [30, 0.70237553],
 [31, 0.63852555],
 [32, 0.7274778],
 [33, 0.66406643],
 [34, 0.6314597],
 [35, 0.6592636],
 [36, 0.6488584],
 [37, 0.6098944],
 [38, 0.6113974],
 [39, 0.5986272],
 [40, 0.5595226],
 [41, 0.70107055],
 [42, 0.67896706],
 [43, 0.65100825],
 [44, 0.7354237],
 [45, 0.6875011],
 [46, 0.677228],
 [47, 0.6510289],
 [48, 0.6786878],
 [49, 0.751333],
 [50, 0.73388004],
 [51, 0.7752007],
 [52, 0.74390614],
 [53, 0.7387235],
 [54, 0.745286],
 [55, 

In [29]:
cos_dis_list = cos_dis.tolist()

In [30]:
from tqdm import tqdm
self_orders=[]

for i in tqdm(range(len(cos_dis))):
    self_num=cos_dis_list[i][i]
    case=cos_dis_list[i]
    new_case=sorted(case,reverse=True)
    self_order=new_case.index(self_num)
    
    self_case=[str(i)+'번째 자기자신 순서와의 유사도 등수',self_order+1]
    self_orders.append(self_case)

100%|██████████| 600/600 [00:00<00:00, 12426.71it/s]


In [31]:
self_orders

[['0번째 자기자신 순서와의 유사도 등수', 557],
 ['1번째 자기자신 순서와의 유사도 등수', 469],
 ['2번째 자기자신 순서와의 유사도 등수', 589],
 ['3번째 자기자신 순서와의 유사도 등수', 207],
 ['4번째 자기자신 순서와의 유사도 등수', 517],
 ['5번째 자기자신 순서와의 유사도 등수', 510],
 ['6번째 자기자신 순서와의 유사도 등수', 563],
 ['7번째 자기자신 순서와의 유사도 등수', 443],
 ['8번째 자기자신 순서와의 유사도 등수', 368],
 ['9번째 자기자신 순서와의 유사도 등수', 494],
 ['10번째 자기자신 순서와의 유사도 등수', 213],
 ['11번째 자기자신 순서와의 유사도 등수', 373],
 ['12번째 자기자신 순서와의 유사도 등수', 394],
 ['13번째 자기자신 순서와의 유사도 등수', 109],
 ['14번째 자기자신 순서와의 유사도 등수', 413],
 ['15번째 자기자신 순서와의 유사도 등수', 412],
 ['16번째 자기자신 순서와의 유사도 등수', 474],
 ['17번째 자기자신 순서와의 유사도 등수', 534],
 ['18번째 자기자신 순서와의 유사도 등수', 354],
 ['19번째 자기자신 순서와의 유사도 등수', 12],
 ['20번째 자기자신 순서와의 유사도 등수', 466],
 ['21번째 자기자신 순서와의 유사도 등수', 324],
 ['22번째 자기자신 순서와의 유사도 등수', 483],
 ['23번째 자기자신 순서와의 유사도 등수', 352],
 ['24번째 자기자신 순서와의 유사도 등수', 486],
 ['25번째 자기자신 순서와의 유사도 등수', 380],
 ['26번째 자기자신 순서와의 유사도 등수', 577],
 ['27번째 자기자신 순서와의 유사도 등수', 92],
 ['28번째 자기자신 순서와의 유사도 등수', 297],
 ['29번째 자기자신 순서와의 유사도 등수', 257],
 ['30번째 자기자신 순서와의 유사도 

In [32]:
self_orders_num = [i[1] for i in self_orders]
np.mean(self_orders_num)

146.43